# Synthefy Forecasting API: A Comprehensive Guide

Note: this notebook requires synthefy-helpers.py to be in the same directory.

Welcome to the Synthefy Forecasting API! This notebook demonstrates how to use our powerful forecasting capabilities to predict time series data with advanced features like context-aware forecasting and external data integration.

## What is Synthefy?

Synthefy is an advanced forecasting API that leverages foundation models to provide accurate predictions for your time series data. Our API offers:

- **Simple Integration**: Easy-to-use Python interface
- **Context-Aware Forecasting**: Incorporate future known data into your predictions
- **External Data Integration**: Leverage weather data for enhanced predictions
- **Metadata Enrichment**: Automatically enrich your data with external metadata that can serve as covariates
- **Flexible Input**: Support for multiple targets and covariates

## Getting Started

First, let's set up our environment and import the necessary libraries.


In [22]:
import pandas as pd
from synthefy_helpers import (
    find_haver_data,
    generate_forecast,
    get_haver_data,
    get_weather_data,
    setup_synthefy_client,
)

# Setup API configuration
X_API_KEY = "XXXX-XXXX"  # Replace with your API key
BASE_URL = "http://prod.synthefy.com"

# Initialize the client
client = setup_synthefy_client(X_API_KEY, BASE_URL)

In [ ]:
# %%
# Load sample data
df = pd.read_csv("walmart.csv")
df["Date"] = pd.to_datetime(df["Date"])
df = df[df["Store"] == "store_1"]  # Use one store for simplicity
df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,store_1,2010-02-05,1643690.90,0,42.31,2.572,211.096358,8.106
1,store_1,2010-02-12,1641957.44,1,38.51,2.548,211.242170,8.106
2,store_1,2010-02-19,1611968.17,0,39.93,2.514,211.289143,8.106
3,store_1,2010-02-26,1409727.59,0,46.63,2.561,211.319643,8.106
4,store_1,2010-03-05,1554806.68,0,46.50,2.625,211.350143,8.106


## Example 1: Basic Forecasting

Let's start with a simple forecasting example using the Walmart dataset.


In [14]:
forecast_results = generate_forecast(
    client=client,
    api_key=X_API_KEY,
    df=df,
    target_columns="Weekly_Sales",
    forecast_timestamps=[
        "2012-11-02",
        "2012-11-09",
        "2012-11-16",
        "2012-11-23",
    ],
    covariate_columns=[],
)

print("📈 Forecasts:")
print(forecast_results["forecast"][["Weekly_Sales", "timestamp"]])

🕐 Auto-detected timestamp column: Date
🎯 Generating forecast for: ['Weekly_Sales']
🚀 Making forecast API call...


✅ Forecast completed successfully!
📈 Generated forecasts for 4 timestamps
📈 Forecasts:
   Weekly_Sales  timestamp
0   1639439.250 2012-11-02
1   1698400.375 2012-11-09
2   1674492.500 2012-11-16
3   2056468.250 2012-11-23


## Example 2: Using Future Known Data

Enhance forecasts by incorporating known future information.


In [15]:
# Split data into historical and future portions
historical_df = df[df["Date"] < pd.Timestamp("2012-09-01")]
future_df = df[df["Date"] >= pd.Timestamp("2012-09-01")]

forecast_results = generate_forecast(
    client=client,
    api_key=X_API_KEY,
    df=historical_df,
    target_columns="Weekly_Sales",
    forecast_timestamps=[ts.isoformat() for ts in future_df["Date"].tolist()],
    covariate_columns=["Holiday_Flag", "CPI"],
    future_df=future_df,
)

print("📈 Forecasts with future data:")
print(forecast_results["forecast"][["Weekly_Sales", "timestamp"]].head())

🕐 Auto-detected timestamp column: Date
🎯 Generating forecast for: ['Weekly_Sales']
📊 Using 2 covariates: ['Holiday_Flag', 'CPI']
🚀 Making forecast API call...


✅ Forecast completed successfully!
📈 Generated forecasts for 8 timestamps
📈 Forecasts with future data:
   Weekly_Sales  timestamp
0   1700102.375 2012-09-07
1   1576326.375 2012-09-14
2   1500093.250 2012-09-21
3   1468684.875 2012-09-28
4   1710045.500 2012-10-05


## Example 3: Fetching Weather Data

Fetch weather data for a given location.


In [16]:
weather_df = get_weather_data(
    client=client,
    api_key=X_API_KEY,
    location_name="Springfield",
    weather_parameters="basic",  # temperature, humidity, precip, wind_speed
    start_time="2012-10-01T00:00:00",
    end_time="2012-10-26T23:59:59",
    auto_select_location=True,  # Skip interactive selection for demo
)

print("✅ Weather data retrieved!")
print("📊 Weather DataFrame:")
print(weather_df.head())

🔍 Searching for location: Springfield (auto-select mode)
✅ Auto-selected: Springfield Lakes (-27.66757, 152.92488)
📊 Using 'basic' parameter preset: ['temperature', 'humidity', 'pressure']
🌤️ Fetching weather data...
✅ Weather data retrieved successfully!
   📊 Added weather columns: ['temperature', 'humidity', 'pressure']
✅ Weather data retrieved!
📊 Weather DataFrame:
   timestamp  temperature  humidity  pressure
0 2012-10-01           19        84      1023
1 2012-10-02           22        73      1027
2 2012-10-03           24        77      1025
3 2012-10-04           29        72      1022
4 2012-10-05           32        61      1020


## Example 4: Enrich Your Data with Weather

In [17]:
walmart_sample = df.head(10)  # Small sample for demo

enriched_df = get_weather_data(
    client=client,
    api_key=X_API_KEY,
    location_name="Springfield",
    weather_parameters="comprehensive",
    user_dataframe=walmart_sample,
    user_timestamp_column="Date",
    auto_select_location=True,  # Skip interactive selection for demo
)

print("📈 Data successfully enriched!")
print("📊 Enriched DataFrame:")
print(enriched_df.head())

🔍 Searching for location: Springfield (auto-select mode)
✅ Auto-selected: Springfield Lakes (-27.66757, 152.92488)
📊 Using 'comprehensive' parameter preset: ['temperature', 'humidity', 'pressure', 'wind_speed', 'precip', 'visibility']
🕐 Auto-detected start time from user data: 2010-02-05T00:00:00
🕐 Auto-detected end time from user data: 2010-04-09T00:00:00
🌤️ Fetching weather data...
✅ Weather data retrieved successfully!
   📊 Added weather columns: ['temperature', 'wind_speed', 'precip', 'humidity', 'visibility', 'pressure']
   📈 Enriched 10 → 10 rows
📈 Data successfully enriched!
📊 Enriched DataFrame:
     Store       Date  Weekly_Sales  Holiday_Flag  Temperature  Fuel_Price  \
0  store_1 2010-02-05    1643690.90             0        42.31       2.572   
1  store_1 2010-02-12    1641957.44             1        38.51       2.548   
2  store_1 2010-02-19    1611968.17             0        39.93       2.514   
3  store_1 2010-02-26    1409727.59             0        46.63       2.561   

## Example 5: Haver Data Search

Find and use Haver's economic indicators to enhance your forecasts.
Haver Analytics provides comprehensive economic and financial datasets.


In [18]:
haver_data = find_haver_data(
    client=client,
    api_key=X_API_KEY,
    prompt="sales consumer spending",
    count=3,
    auto_select=True,  # Auto-select for demo
)

print("✅ Found Haver data!")
print(f"📊 Series: {haver_data.get('name')}@{haver_data.get('database_name')}")
print(f"📝 Description: {haver_data.get('description')}")

🔍 Finding Haver data for: 'sales consumer spending'
🤖 Auto-selected: Sales: Total Business (SA, Bil.$)
   📊 TSSTB@USECON
✅ Found Haver data!
📊 Series: TSSTB@USECON
📝 Description: Sales: Total Business (SA, Bil.$)


## Example 6: Fetch Haver Data

Haver Analytics provides comprehensive economic and financial time series data.


In [19]:
haver_df = get_haver_data(
    client=client,
    api_key=X_API_KEY,
    search_prompt="unemployment rate",
    start_time="2010-01-01T00:00:00",
    end_time="2013-01-01T00:00:00",
    auto_select=True,
)

print("📋 Haver Data DataFrame:")
print(haver_df.head())
print(
    f"📈 Data Range: {haver_df['timestamp'].min()} to {haver_df['timestamp'].max()}"
)

🔍 Finding Haver data with prompt: 'unemployment rate'
🔍 Finding Haver data for: 'unemployment rate'
🤖 Auto-selected: Unemployment Rate: < HS Diploma: 25+ Years (SA, %)
   📊 LRLHS25@USECON
📊 Using found data: LRLHS25@USECON
🏦 Fetching Haver data: LRLHS25@USECON
📈 Fetching Haver data...
✅ Haver data retrieved successfully!
   📊 Added Haver columns: ['value']
📋 Haver Data DataFrame:
   timestamp  LRLHS25
0 2010-01-01     15.3
1 2010-02-01     15.8
2 2010-03-01     14.9
3 2010-04-01     14.7
4 2010-05-01     14.6
📈 Data Range: 2010-01-01 00:00:00 to 2013-01-01 00:00:00


## Example 7: Forecast with Enriched Data

First enrich your data, then forecast with the enriched DataFrame.


In [21]:
# Step 1: Enrich data with weather
enriched_df = get_weather_data(
    client=client,
    api_key=X_API_KEY,
    location_name="Springfield",
    weather_parameters="basic",
    user_dataframe=df.head(20),
    user_timestamp_column="Date",
    auto_select_location=True,
)

print("✅ Data enriched with weather!")
print(enriched_df.head())

# Step 2: Get weather column names for covariates
metadata_columns = [col for col in enriched_df.columns if col not in df.columns]
print(f"Added metadata columns: {metadata_columns}")

# Step 3: Generate forecast with enriched data
forecast_results = generate_forecast(
    client=client,
    api_key=X_API_KEY,
    df=enriched_df,
    target_columns="Weekly_Sales",
    forecast_timestamps=[
        "2012-11-02",
        "2012-11-09",
        "2012-11-16",
        "2012-11-23",
    ],
    covariate_columns=["Holiday_Flag", "CPI"] + metadata_columns,
)

print("📈 Forecast with metadata:")
print(forecast_results["forecast"][["Weekly_Sales", "timestamp"]])

🔍 Searching for location: Springfield (auto-select mode)
✅ Auto-selected: Springfield Lakes (-27.66757, 152.92488)
📊 Using 'basic' parameter preset: ['temperature', 'humidity', 'pressure']
🕐 Auto-detected start time from user data: 2010-02-05T00:00:00
🕐 Auto-detected end time from user data: 2010-06-18T00:00:00
🌤️ Fetching weather data...


✅ Weather data retrieved successfully!
   📊 Added weather columns: ['temperature', 'humidity', 'pressure']
   📈 Enriched 20 → 20 rows
✅ Data enriched with weather!
     Store       Date  Weekly_Sales  Holiday_Flag  Temperature  Fuel_Price  \
0  store_1 2010-02-05    1643690.90             0        42.31       2.572   
1  store_1 2010-02-12    1641957.44             1        38.51       2.548   
2  store_1 2010-02-19    1611968.17             0        39.93       2.514   
3  store_1 2010-02-26    1409727.59             0        46.63       2.561   
4  store_1 2010-03-05    1554806.68             0        46.50       2.625   

          CPI  Unemployment  temperature  humidity  pressure  
0  211.096358         8.106           29        82      1011  
1  211.242170         8.106           30        81      1014  
2  211.289143         8.106           25        82      1019  
3  211.319643         8.106           26        85      1020  
4  211.350143         8.106           23        94  

## Summary

This notebook demonstrated the key capabilities of the Synthefy Forecasting API:

1. **Basic Forecasting**: Simple time series prediction
2. **Future Known Data**: Using known future values as context
3. **Weather Data**: Fetching and enriching with weather information
4. **Haver Data**: Economic indicators for enhanced forecasting
5. **Combined Forecasting**: Using enriched data for improved predictions

## Next Steps

- Replace `X_API_KEY` with your actual API key
- Try with your own datasets
- Experiment with different weather parameters and locations
- Explore various Haver economic indicators
- Combine multiple metadata sources for enhanced accuracy


